## Problem Statement

This project aims to understand the prevalence of standardized test taking to California high school students. Some questions I'd like to consider are 
* Which schools have the highest testing rates?
* Do schools where more students take standardized tests also score better on those tests?
* Do different tests (ACT, SAT, potentially AP) have different patterns?
* How did testing prevalence / scores change in California during COVID?

### Contents

## Background

**To-Do:**

### Datasets Used

**To-Do**

### Outside Research

**To-Do:** 

### Cali - ACTs

In [151]:
# Imports:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### View Dataset

In [152]:
act_raw = pd.read_csv('data/act_2019_ca.csv')

In [153]:
act_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CDS          2309 non-null   float64
 1   CCode        2309 non-null   float64
 2   CDCode       2309 non-null   float64
 3   SCode        1787 non-null   float64
 4   RType        2309 non-null   object 
 5   SName        1729 non-null   object 
 6   DName        2251 non-null   object 
 7   CName        2309 non-null   object 
 8   Enroll12     2309 non-null   float64
 9   NumTstTakr   2309 non-null   float64
 10  AvgScrRead   1953 non-null   object 
 11  AvgScrEng    1953 non-null   object 
 12  AvgScrMath   1953 non-null   object 
 13  AvgScrSci    1953 non-null   object 
 14  NumGE21      1953 non-null   object 
 15  PctGE21      1953 non-null   object 
 16  Year         2309 non-null   object 
 17  Unnamed: 17  0 non-null      float64
dtypes: float64(7), object(11)
memory usage: 325.0+ K

In [154]:
act_raw.head()

,CDS,CCode,CDCode,SCode,RType,SName,DName,CName,Enroll12,NumTstTakr,AvgScrRead,AvgScrEng,AvgScrMath,AvgScrSci,NumGE21,PctGE21,Year,Unnamed: 17
0,3.366993e+13,33.0,3366993.0,129882.0,S,21st Century Learning Institute,Beaumont Unified,Riverside,18.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,NaN
1,1.964212e+13,19.0,1964212.0,1995596.0,S,ABC Secondary (Alternative),ABC Unified,Los Angeles,58.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,NaN
2,1.563776e+13,15.0,1563776.0,1530377.0,S,Abraham Lincoln Alternative,Southern Kern Unified,Kern,18.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,NaN
3,4.369666e+13,43.0,4369666.0,4333795.0,S,Abraham Lincoln High,San Jose Unified,Santa Clara,463.0,53.0,23,22,22,23,34,64.15,2018-19,NaN
4,1.964733e+13,19.0,1964733.0,1935121.0,S,Abraham Lincoln Senior High,Los Angeles Unified,Los Angeles,226.0,19.0,21,20,23,22,11,57.89,2018-19,NaN


### Data Cleaning

In [155]:
act_ca = act_raw.drop(columns='Unnamed: 17') # This column is all NA

In [156]:
act_ca['PctGE21'].value_counts()

*        532
50.00     22
33.33     10
0.00       9
55.56      9
        ... 
6.56       1
8.00       1
80.81      1
24.53      1
57.58      1
Name: PctGE21, Length: 915, dtype: int64

#### Drop unused columns
For our analysis we won't use the codes (except CDS for identification), school/district/county names, or any of the 4 subject scores

In [157]:
act_ca.columns

Index(['CDS', 'CCode', 'CDCode', 'SCode', 'RType', 'SName', 'DName', 'CName',
       'Enroll12', 'NumTstTakr', 'AvgScrRead', 'AvgScrEng', 'AvgScrMath',
       'AvgScrSci', 'NumGE21', 'PctGE21', 'Year'],
      dtype='object')

In [158]:
code_cols = ['CCode', 'CDCode', 'SCode']
name_cols = ['SName', 'DName', 'CName']
subject_cols = ['AvgScrRead', 'AvgScrEng', 'AvgScrMath', 'AvgScrSci']
dropped_cols = code_cols + name_cols + subject_cols
act_ca.drop(columns=dropped_cols, inplace = True)

#### Exclude schools with very low sample size

In [159]:
act_ca = act_ca[act_ca['PctGE21'] != '*'] 

In [160]:
act_ca.loc[act_ca['NumGE21'].isna(), ['NumGE21']] = 0 # NaN means no students took test

In [161]:
act_raw.sort_values('Enroll12').head()

,CDS,CCode,CDCode,SCode,RType,SName,DName,CName,Enroll12,NumTstTakr,AvgScrRead,AvgScrEng,AvgScrMath,AvgScrSci,NumGE21,PctGE21,Year,Unnamed: 17
1103,2.000000e+12,2.0,0.0,0.0,C,NaN,NaN,Alpine,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,NaN
1854,1.964733e+13,19.0,1964733.0,133280.0,S,PUC Nueva Esperanza Charter Academy,Los Angeles Unified,Los Angeles,0.0,22.0,16,15,17,16,1,4.55,2018-19,NaN
238,3.768213e+13,37.0,3768213.0,3730934.0,S,Campo High (Continuation),Mountain Empire Unified,San Diego,3.0,1.0,*,*,*,*,*,*,2018-19,NaN
336,3.166944e+13,31.0,3166944.0,3130192.0,S,Cold Stream Alternative,Tahoe-Truckee Unified,Placer,3.0,1.0,*,*,*,*,*,*,2018-19,NaN
314,3.768338e+13,37.0,3768338.0,124347.0,S,City Heights Preparatory Charter,San Diego Unified,San Diego,4.0,1.0,*,*,*,*,*,*,2018-19,NaN


Alpine county (https://en.wikipedia.org/wiki/Alpine_County%2C_California) population 1129 has no high school students in the county so we drop it.

In [162]:
act_ca.drop(index = 1103, inplace=True)

##### Should we drop rows with no students taking test?
These are wanted for particpation metrics but not wanted for performance metrics

In [163]:
act_ca['PctGE21'].isna().sum()
act_ca['PctGE21'].value_counts(dropna = False)

NaN      356
50.00     22
33.33     10
55.56      9
0.00       9
        ... 
6.56       1
8.00       1
80.81      1
24.53      1
57.58      1
Name: PctGE21, Length: 915, dtype: int64

#### School, District, County counts

In [164]:
act_ca['RType'].value_counts()

S    1308
D     413
C      54
X       1
Name: RType, dtype: int64

In [165]:
act_ca[['RType', 'PctGE21']].groupby('RType').count()

,PctGE21
RType,
C,54
D,350
S,1016
X,1


#### Fix data types

In [166]:
act_ca.dtypes

CDS           float64
RType          object
Enroll12      float64
NumTstTakr    float64
NumGE21        object
PctGE21        object
Year           object
dtype: object

* Make Enroll12 and NumTstTakr ints
* Make NumGE21 int
* Make PctGE21 a float

In [167]:
act_ca[act_ca['Enroll12'].isna()]

,CDS,RType,Enroll12,NumTstTakr,NumGE21,PctGE21,Year
2309,NaN,NaN,NaN,NaN,0,NaN,NaN


In [168]:
act_ca.drop(index=2309, inplace = True)

In [169]:
act_ca['Enroll12'].astype(int)

0        18
1        58
2        18
3       463
4       226
       ... 
2302    138
2303    394
2305    102
2306    628
2308     47
Name: Enroll12, Length: 1776, dtype: int32

In [170]:
act_ca['NumTstTakr'].astype(int)

0        0
1        0
2        0
3       53
4       19
        ..
2302    38
2303    56
2305     0
2306    61
2308     0
Name: NumTstTakr, Length: 1776, dtype: int32

In [171]:
act_ca['Enroll12'] = act_ca['Enroll12'].astype(int)
act_ca['NumTstTakr'] = act_ca['NumTstTakr'].astype(int)

In [172]:
act_ca['NumGE21'] = act_ca['NumGE21'].astype(int)
act_ca['PctGE21'] = act_ca['PctGE21'].astype(float)

### Features to measure test performance and participation
Since our goal is to track test performance and particpation we need a metric for each of these.
For our purposes, we can measure the as follows: 
* perfomance by the percentage of test takers with a composite score above 21
* participation by the percentage of enrollees who take the test

In [173]:
act_ca['ACT_taken_pct'] = 100*act_ca['NumTstTakr']/act_ca['Enroll12']
# act_ca['ACT_high_score_%'] = 100*act_ca['NumGE21']/act_ca['NumTstTakr'] # <-- This is redundant with PctGE21

Some schools have more test takers than students.
* 12 are alliance charter schools in LA. https://www.laalliance.org/
* 9 Aspire charter schools https://aspirepublicschools.org/discover_aspire/
* Most schools are in cities / urban areas - exception Los Molinos High

This could be due to homeschooling or students attending other schools that don't proctor the exam taking the exam at a larger schools.
For my purposes to capture the significance of testing to a school I will allow the percent to go above 100 but to handle outliers I limit it at 125%

In [181]:
act_raw.query('Enroll12 < NumTstTakr').drop(columns = code_cols + subject_cols + ['Unnamed: 17', 'Year'])

,CDS,RType,SName,DName,CName,Enroll12,NumTstTakr,NumGE21,PctGE21
34,1.964733e+13,S,Alliance Alice M. Baxter College-Ready High,Los Angeles Unified,Los Angeles,61.0,77.0,12,15.58
35,1.964733e+13,S,Alliance Cindy and Bill Simon Technology Acade...,Los Angeles Unified,Los Angeles,123.0,155.0,13,8.39
39,1.964733e+13,S,Alliance Judy Ivie Burton Technology Academy High,Los Angeles Unified,Los Angeles,138.0,162.0,24,14.81
40,1.964733e+13,S,Alliance Leichtman-Levine Family Foundation En...,Los Angeles Unified,Los Angeles,128.0,155.0,40,25.81
41,1.964733e+13,S,Alliance Marc & Eva Stern Math and Science,Los Angeles Unified,Los Angeles,141.0,157.0,48,30.57
42,1.964733e+13,S,Alliance Margaret M. Bloomfield Technology Aca...,Los Angeles Unified,Los Angeles,114.0,139.0,19,13.67
43,1.964733e+13,S,Alliance Morgan McKinzie High,Los Angeles Unified,Los Angeles,73.0,86.0,7,8.14
44,1.964733e+13,S,Alliance Ouchi-O'Donovan 6-12 Complex,Los Angeles Unified,Los Angeles,130.0,148.0,16,10.81
45,1.964733e+13,S,Alliance Patti And Peter Neuwirth Leadership A...,Los Angeles Unified,Los Angeles,131.0,164.0,18,10.98
46,1.964733e+13,S,Alliance Piera Barbaglia Shaheen Health Servic...,Los Angeles Unified,Los Angeles,105.0,143.0,4,2.80


In [175]:
act_ca['ACT_taken_pct'].clip(upper = 125, inplace = True)

#### Select Data for Later Analysis

In [185]:
# act_ca.head()

In [184]:
act_df = act_ca.drop(columns = ['NumTstTakr', 'NumGE21'])
act_df.head()

,CDS,RType,Enroll12,PctGE21,Year,ACT_taken_pct
0,3.366993e+13,S,18,NaN,2018-19,0.000000
1,1.964212e+13,S,58,NaN,2018-19,0.000000
2,1.563776e+13,S,18,NaN,2018-19,0.000000
3,4.369666e+13,S,463,64.15,2018-19,11.447084
4,1.964733e+13,S,226,57.89,2018-19,8.407080


In [186]:
act_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1776 entries, 0 to 2308
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CDS            1776 non-null   float64
 1   RType          1776 non-null   object 
 2   Enroll12       1776 non-null   int32  
 3   PctGE21        1421 non-null   float64
 4   Year           1776 non-null   object 
 5   ACT_taken_pct  1776 non-null   float64
dtypes: float64(3), int32(1), object(2)
memory usage: 90.2+ KB


In [193]:
act_ca['RType'].value_counts()

S    1308
D     413
C      54
X       1
Name: RType, dtype: int64

### Cali SATs

#### View Dataset

In [ ]:
sat_ca = pd.read_csv('../data/sat_2019_ca.csv')

In [ ]:
sat_ca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2580 entries, 0 to 2579
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CDS                    2579 non-null   float64
 1   CCode                  2579 non-null   float64
 2   CDCode                 2579 non-null   float64
 3   SCode                  2579 non-null   float64
 4   RType                  2579 non-null   object 
 5   SName                  1982 non-null   object 
 6   DName                  2521 non-null   object 
 7   CName                  2579 non-null   object 
 8   Enroll12               2579 non-null   float64
 9   NumTSTTakr12           2579 non-null   float64
 10  NumERWBenchmark12      2304 non-null   object 
 11  PctERWBenchmark12      2304 non-null   object 
 12  NumMathBenchmark12     2304 non-null   object 
 13  PctMathBenchmark12     2304 non-null   object 
 14  Enroll11               2579 non-null   float64
 15  NumT

In [ ]:
sat_ca.head()

,CDS,CCode,CDCode,SCode,RType,SName,DName,CName,Enroll12,NumTSTTakr12,...,NumERWBenchmark11,PctERWBenchmark11,NumMathBenchmark11,PctMathBenchmark11,TotNumBothBenchmark12,PctBothBenchmark12,TotNumBothBenchmark11,PctBothBenchmark11,Year,Unnamed: 25
0,6.615981e+12,6.0,661598.0,630046.0,S,Colusa Alternative Home,Colusa Unified,Colusa,18.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,NaN
1,6.616061e+12,6.0,661606.0,634758.0,S,Maxwell Sr High,Maxwell Unified,Colusa,29.0,10.0,...,*,*,*,*,*,*,*,*,2018-19,NaN
2,1.964733e+13,19.0,1964733.0,1930924.0,S,Belmont Senior High,Los Angeles Unified,Los Angeles,206.0,102.0,...,42,24.14,12,6.90,14,13.73,11,6.32,2018-19,NaN
3,1.964733e+13,19.0,1964733.0,1931476.0,S,Canoga Park Senior High,Los Angeles Unified,Los Angeles,227.0,113.0,...,97,35.27,37,13.45,18,15.93,35,12.73,2018-19,NaN
4,1.964733e+13,19.0,1964733.0,1931856.0,S,Whitman Continuation,Los Angeles Unified,Los Angeles,18.0,14.0,...,*,*,*,*,*,*,*,*,2018-19,NaN


#### Clean Data

In [ ]:
sat_ca.drop(columns='Unnamed: 25', inplace=True)

In [ ]:
sat_ca.head()

,CDS,CCode,CDCode,SCode,RType,SName,DName,CName,Enroll12,NumTSTTakr12,...,NumTSTTakr11,NumERWBenchmark11,PctERWBenchmark11,NumMathBenchmark11,PctMathBenchmark11,TotNumBothBenchmark12,PctBothBenchmark12,TotNumBothBenchmark11,PctBothBenchmark11,Year
0,6.615981e+12,6.0,661598.0,630046.0,S,Colusa Alternative Home,Colusa Unified,Colusa,18.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19
1,6.616061e+12,6.0,661606.0,634758.0,S,Maxwell Sr High,Maxwell Unified,Colusa,29.0,10.0,...,6.0,*,*,*,*,*,*,*,*,2018-19
2,1.964733e+13,19.0,1964733.0,1930924.0,S,Belmont Senior High,Los Angeles Unified,Los Angeles,206.0,102.0,...,174.0,42,24.14,12,6.90,14,13.73,11,6.32,2018-19
3,1.964733e+13,19.0,1964733.0,1931476.0,S,Canoga Park Senior High,Los Angeles Unified,Los Angeles,227.0,113.0,...,275.0,97,35.27,37,13.45,18,15.93,35,12.73,2018-19
4,1.964733e+13,19.0,1964733.0,1931856.0,S,Whitman Continuation,Los Angeles Unified,Los Angeles,18.0,14.0,...,5.0,*,*,*,*,*,*,*,*,2018-19


In [ ]:
sat_ca[sat_ca['TotNumBothBenchmark12'].isna()]

,CDS,CCode,CDCode,SCode,RType,SName,DName,CName,Enroll12,NumTSTTakr12,...,NumTSTTakr11,NumERWBenchmark11,PctERWBenchmark11,NumMathBenchmark11,PctMathBenchmark11,TotNumBothBenchmark12,PctBothBenchmark12,TotNumBothBenchmark11,PctBothBenchmark11,Year
0,6.615981e+12,6.0,661598.0,630046.0,S,Colusa Alternative Home,Colusa Unified,Colusa,18.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19
12,1.563776e+13,15.0,1563776.0,1530377.0,S,Abraham Lincoln Alternative,Southern Kern Unified,Kern,18.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19
19,1.062117e+13,10.0,1062117.0,1030469.0,S,Enterprise Alternative,Clovis Unified,Fresno,18.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19
36,3.768163e+13,37.0,3768163.0,137109.0,S,Diego Valley East Public Charter,Julian Union Elementary,San Diego,78.0,0.0,...,1.0,*,*,*,*,NaN,NaN,*,*,2018-19
43,3.467314e+13,34.0,3467314.0,3430352.0,S,Las Flores High (Alternative),Elk Grove Unified,Sacramento,64.0,0.0,...,1.0,*,*,*,*,NaN,NaN,*,*,2018-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540,4.110413e+13,41.0,4110413.0,0.0,D,NaN,San Mateo County Office of Education,San Mateo,97.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19
2561,1.976992e+13,19.0,1976992.0,0.0,D,NaN,SBE - Prepa Tec Los Angeles High,Los Angeles,0.0,0.0,...,47.0,17,36.17,4,8.51,NaN,NaN,4,8.51,2018-19
2572,5.071092e+13,50.0,5071092.0,0.0,D,NaN,Hart-Ransom Union Elementary,Stanislaus,18.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19
2573,5.071134e+13,50.0,5071134.0,0.0,D,NaN,Keyes Union,Stanislaus,25.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19


In [ ]:
sat_ca.loc[:,['NumERWBenchmark12', 'NumERWBenchmark11', 
            'NumMathBenchmark12', 'NumMathBenchmark11', 
            'TotNumBothBenchmark12', 'TotNumBothBenchmark11']].fillna(0, inplace = True)

In [ ]:
sat_schools = set(sat_ca['SCode'].dropna())
# sat_schools

NameError: name 'sat_ca' is not defined

In [ ]:
sat_ca.loc[sat_ca['RType'] == 'S', ['Enroll12', 'TotNumBothBenchmark12']]

,Enroll12,TotNumBothBenchmark12
0,18.0,NaN
1,29.0,*
2,206.0,14
3,227.0,18
4,18.0,*
...,...,...
1976,76.0,6
1977,15.0,*
1978,27.0,*
1979,1083.0,293


### Compare schools

In [ ]:
schools = act_schools & sat_schools
act_only_schools = act_schools - sat_schools
sat_only_schools = sat_schools - act_schools
print(len(schools))
print(len(act_only_schools))
print(len(sat_only_schools))

1306
3
676
